In [1]:
from __future__ import print_function
import json
import pandas as pd
#%matplotlib inline
import matplotlib.pyplot as mpp
import seaborn as sns
from ezprettyjson import prettyjson
from ml.logutils import loadall, detvalues
from ml.processing import *
from itertools import dropwhile
import numpy as np
from cycler import cycler
from numpy.random import randint, normal
from scipy.stats import norm, shapiro, anderson
import gzip, cPickle

In [2]:
# old: catalog-merged-fixed
with open('catalog-modern_only-merged.json', 'r') as f:
    catj = json.load(f)

In [3]:
not_saved = []
def show(name = None):
    if name and not name in not_saved:
        sns.plt.savefig(name+'.png')
        sns.plt.clf()
    else:
        sns.plt.show()

In [4]:
prettyjson(catj)

In [5]:
def get(key):
    if isinstance(key, list):
        return (lambda x: [x[k] for k in key])
    else:
        return (lambda x: x[key])

def put(key, value):
    if isinstance(key, list):
        return (lambda x: dict(x, **{key[i]: value[i] for i in xrange(len(key))}))
    else:
        return (lambda x: dict(x, **{key: value}))
    
def only(keys):
    return (lambda x: {k: v for k,v in x.iteritems() if k in keys})

def drop(key):
    if isinstance(key, list):
        return (lambda x: {k: v for k,v in x.iteritems() if not k in key})
    else:
        return (lambda x: {k: v for k,v in x.iteritems() if k != key})

def on(key, fun):
    return (lambda x: put(key, fun(x[key]))(x))

def over(key, fun):
    return (lambda x: put(key, dmap(fun, x[key]))(x) 
                      if isinstance(x[key], dict) else (
                          put(key, map(fun, x[key]))(x)
                          if isinstance(x[key], list) else on(key, fun)(x)))

def sift(fun, items = False):
    if items:
        return (lambda x: {k: v for k,v in x.iteritems() if fun(k,v)})
    else:
        return (lambda x: {k: v for k,v in x.iteritems() if fun(v)})

def chain(*funs):
    return (lambda x: reduce(lambda p,c: c(p), funs, x))

def liftup(key):
    def internal(x):
        if isinstance(x[key], dict):
            return dict(drop(key)(x), **{str(key)+'_'+str(k): v for k,v in x[key].iteritems()})
        elif isinstance(x[key], list):
            return dict(drop(key)(x), **{str(key)+'_'+str(i): x[key][i] for i in xrange(len(x[key]))})
    return internal

def rec_liftup(key, also_lists = False):
    return (lambda x: dict(drop(key)(x), **reduce(lambda p, c: rec_liftup(c)(p) 
                                                               if isinstance(p[c], dict) or 
                                                                  (also_lists and isinstance(p[c], list))
                                                               else p,
                                                  liftup(key)({key: x[key]}).iterkeys(),
                                                  liftup(key)({key: x[key]}))))

def combine(sources, fun, destination, keep = False):
    dest_less_srcs = sources+[]
    if destination in dest_less_srcs:
        dest_less_srcs.remove(destination)
    return (lambda x: chain(put(destination, 
                                fun(*get(sources)(x))), 
                            drop(dest_less_srcs if not keep else []))(x))

def nmap(fun, coll):
    return map(lambda e: map(fun, e), coll)

def dmap(fun, dct = None):
    if not dct is None:
        return { k: fun(v) for k,v in dct.iteritems() }
    else:
        return (lambda x: dmap(fun, x))

def keymap(fun, dct = None):
    if not dct is None:
        return { fun(k): v for k,v in dct.iteritems() }
    else:
        return (lambda x: keymap(fun, x))

def smart_combine(old, new):
    if isinstance(old, list):
        if isinstance(new, list):
            return old + new
        else:
            return old + [new]
    elif isinstance(old, dict):
        if not all(ok in new.keys() for ok in old.iterkeys()):
            return old
        return { k: smart_combine(old[k], new[k]) 
                 for k in old.iterkeys() }
    else:
        return [old, new]

def columnify(old_keys = [], type_col = "", value_col = "", match_length = True):
    tc = type_col or "type"
    vc = value_col or "value"
    def internal(dct):
        ok = old_keys or list(sorted(dct.iterkeys()))
        if match_length:
            vals = get(ok)(dct)
            lens = map(lambda e: len(e) if isinstance(e, list) or isinstance(e, dict) else 1, vals)
        else:
            lens = [1 for _ in ok]
        res = combine(ok, lambda *ov: reduce(smart_combine, ov), vc)(dct)
        res[tc] = [ok[i] for i in xrange(len(ok)) for _ in xrange(lens[i])]
        return res
    return internal

def flip(dct):
    assert all(isinstance(v, dict) for v in dct.itervalues())
    return {ik: {ok: dct[ok][ik] for ok in dct.iterkeys()} for ik in dct.values()[0].iterkeys()}

def dffd(dct):
    return pd.DataFrame.from_dict(dct)

def trace(x):
    print(x)
    return x

def iden(x):
    return x

In [6]:
on('bla', lambda i: i*2)(rec_liftup('test')(over('bla', lambda i: i+1)({'test': {'a': {'yo': 0, 'hi': 1}, 'b': 2}, 'bla': [3, 4]})))
combine(['hi','yo'], lambda h, y: h+y, 'stuff')({'hi': 1, 'yo': 2})
columnify(['a','b','c'], 'abc-type', 'abc-value')({'a': [1,3], 'b': [2,5], 'c': [3,6], 'd': [0,0,0]})
columnify()({'a': [1,3], 'b': [2,5], 'c': [3,6], 'd': [0,0,0]})
#keymap(len, {'a': 1, 'aa': 2, 'aaa': 3})
rec_liftup('a', also_lists = True)({'a': {'b': [1,2,3,4], 'c': {1: 2, 3: 4}}})
#rec_liftup('a')({'a': [1,2,3,4]})

{'a_b_0': 1, 'a_b_1': 2, 'a_b_2': 3, 'a_b_3': 4, 'a_c_1': 2, 'a_c_3': 4}

In [7]:
# Fix names
catj = keymap(lambda k: k.replace('-2',''), catj)
# Split into categories
eds = {}
mods = {}
res = {}
for k, v in catj.iteritems():
    if k.startswith('e-'):
        eds[k[2:]] = v
    elif k.startswith('m-'):
        mods[k[2:]] = v
    else:
        res[k] = v
# Per experiment
experiments = list(sorted(res.iterkeys()))
pe_eds, pe_mods = map(lambda x: {e: {'-'.join(k.split('-')[-2:]): v 
                                     for k,v in x.iteritems() if k.startswith(e+'-x')}
                                 for e in experiments},
                      [eds, mods])
# Per person
# First make a version where the x_i get replaced by the actual names, otherwise confusion occurs
name_mapping = {'-'.join(spl[:-1])+'-': '-'.join(spl[:-2])+'-'+res['-'.join(spl[:-2])]['cross_validation']['drops'][int(spl[-2][1:])]+'-'
                for k in list(sorted(eds.iterkeys()))
                for spl in (k.split('-'),)}
pp_eds = keymap(lambda k: [k.replace(nmk, nmv) 
                           for nmk, nmv in name_mapping.iteritems()
                           if k.startswith(nmk)][0], 
                eds)
name_mapping = {'-'.join(spl[:-1])+'-': '-'.join(spl[:-2])+'-'+res['-'.join(spl[:-2])]['cross_validation']['drops'][int(spl[-2][1:])]+'-'
                for k in list(sorted(mods.iterkeys()))
                for spl in (k.split('-'),)}
pp_mods = keymap(lambda k: [k.replace(nmk, nmv) 
                            for nmk, nmv in name_mapping.iteritems()
                            if k.startswith(nmk)][0], 
                 mods)
pp_eds, pp_mods = map(lambda x: {n: {'-'.join(k.split('-')[2:-2]): v
                                     for k,v in x.iteritems() if k.split('-')[-2] == n}
                                 # For old data
                                 #for n in ["wannes","lawrence","zerzerzer","Wassie","Jeroen","joschout","cristaline","els","noottein"]},
                                 for n in ["wannes", "lawrence", "zerzerzer", "Wassie", "Jeroen", "joschout", "cristaline", "emilio", "els", "noottein", "andres"]},
                      [pp_eds, pp_mods])

In [8]:
def anonymize(name):
    return {'wannes': 'B'
           ,'lawrence': 'L'
           ,'zerzerzer': 'Z'
           ,'Wassie': 'W'
           ,'Jeroen': 'J'
           ,'joschout': 'S'
           ,'cristaline': 'C'
           ,'emilio': 'E'
           ,'els': 'M'
           ,'noottein': 'N'
           ,'andres': 'A'}[name]

In [9]:
prettyjson(pp_eds)

# Encdecs

In [30]:
# Accuracy per encdec layer for all experiments
sns.boxplot(data = dffd(reduce(smart_combine, detvalues(dmap(chain(get('layers'), lambda d: dmap(get('accuracy'), d)), eds)))))
sns.plt.title('Nauwkeurigheid per laag voor alle autoencoder lagen')
sns.plt.xlabel('Laag-nummer')
sns.plt.ylabel('Nauwkeurigheid')
sns.plt.tight_layout()
show('encdecs-layers-accuracy')

In [141]:
# Layer accuracy split out according to noise parameters
sns.set_context('notebook', font_scale = 1)
fp = sns.factorplot(data = dffd(reduce(smart_combine, 
                                       detvalues(dmap(chain(drop('layer_sizes'), 
                                                            over('layers', 
                                                            drop('loss')),
                                                            combine(['gaussian_base_sigma','enc_use_noise'],
                                                                    lambda bs, un: bs if un else 0,
                                                                    'base_sigma',
                                                                    keep = True),
                                                            only(['base_sigma', 'layers']),
                                                            combine(['layers'], iden, 'Laag'),
                                                            rec_liftup('Laag'),
                                                            keymap(lambda k: k.replace('_',' ')),
                                                            keymap(lambda k: k.replace('base s', 'S')),
                                                            keymap(lambda k: k.replace(' accuracy',''))), 
                                                      sift(lambda v: v['gaussian_sigma_factor'] == 1)(eds))))),
                    col = 'Sigma',
                    kind = 'box',
                    aspect = 0.8,
                    order = ['Laag 0', 'Laag 1', 'Laag 2'])
sns.plt.suptitle('Nauwkeurigheid van autoencoder lagen per instelling van de sigma van de toegevoegde ruis')
sns.plt.subplots_adjust(left=0.02, bottom=0.10, right=0.99, top=0.88, wspace=0.05, hspace=0.2)
show('encdecs-noise-layers_accuracy')

In [143]:
# Layer accuracy split out according to epochs
sns.set_context('notebook', font_scale = 1)
sns.factorplot(data = dffd(reduce(smart_combine, 
                                  detvalues(dmap(chain(drop('layer_sizes'), 
                                                       over('layers', drop('loss')),
                                                       only(['layers', 'epochs']),
                                                       combine(['layers'], iden, 'Laag'),
                                                       combine(['epochs'], iden, 'Iteraties'),
                                                       rec_liftup('Laag'),
                                                       keymap(lambda k: k.replace('_',' ')),
                                                       keymap(lambda k: k.replace(' accuracy',''))), 
                                                 eds)))),
               col = 'Iteraties',
               kind = 'box',
               aspect = 0.8,
               order = ['Laag 0', 'Laag 1', 'Laag 2'])
sns.plt.suptitle('Nauwkeurigheid van autoencoder lagen per instelling van het aantal iteraties')
sns.plt.subplots_adjust(left=0.02, bottom=0.10, right=0.99, top=0.88, wspace=0.05, hspace=0.2)
#sns.plt.show()
show('encdecs-epochs-layer_accuracy')

In [137]:
# Select data
la_for_exp = dmap(lambda ev: reduce(smart_combine,
                                    detvalues(dmap(chain(over('layers', drop('loss')),
                                                         #on('layers', drop('0')),
                                                         only(['layers']), 
                                                         rec_liftup('layers'),
                                                         keymap(lambda k: k.replace('layers_','').replace('_accuracy','')),
                                                         columnify()), 
                                                   ev))), 
                  pe_eds)
# Get everything into the correct columns
la_for_exp = keymap(lambda s: '-'.join(s.split('-')[2:]), la_for_exp)
exps = list(sorted(la_for_exp.keys()))
la_for_exp = reduce(lambda p,c: liftup(c)(p),
                    la_for_exp.iterkeys(),
                    la_for_exp)
la_for_exp = chain(columnify([e+'_type' for e in exps], 'Experiment', 'Laag'),
                   columnify([e+'_value' for e in exps], 'Experiment', 'Nauwkeurigheid'),
                   over('Experiment', lambda n: ''.join(n.split('_')[:-1])))(la_for_exp)
# Plot!
sns.set_context('notebook', font_scale = 1.5)
p = sns.factorplot(data = dffd(la_for_exp),
                   col = 'Laag',
                   y = 'Nauwkeurigheid',
                   x = 'Experiment',
                   kind = 'box',
                   size = 10,
                   aspect = 0.5)
sns.plt.suptitle('Nauwkeurigheid van autoencoder lagen per experiment')
p.set_xticklabels(rotation=90)
sns.plt.tight_layout()
sns.plt.subplots_adjust(top=0.92)
show('encdecs-experiment-layer_accuracy')

In [38]:
# Accuracy vs Train-accuracy. Maybe not so useful?
# Select data
la_for_exp = dmap(lambda ev: reduce(smart_combine,
                                    detvalues(dmap(chain(only(['quality', 'train_quality']), 
                                                         columnify()), 
                                                   ev))), 
                  pe_eds)
# Get everything into the correct columns
la_for_exp = keymap(lambda s: '-'.join(s.split('-')[2:]), la_for_exp)
exps = list(sorted(la_for_exp.keys()))
la_for_exp = reduce(lambda p,c: liftup(c)(p),
                    la_for_exp.iterkeys(),
                    la_for_exp)
la_for_exp = chain(columnify([e+'_type' for e in exps], 'Experiment', 'Resultaattype'),
                   columnify([e+'_value' for e in exps], 'Experiment', 'Waarde'),
                   over('Experiment', lambda n: ''.join(n.split('_')[:-1])))(la_for_exp)
# Plot!
#p = sns.factorplot(data = dffd(la_for_exp),
#                   col = 'Resultaattype',
#                   y = 'Waarde',
#                   x = 'Experiment',
#                   kind = 'box')
#p.set_xticklabels(rotation=30)
#sns.plt.show()

In [39]:
# Select data
la_for_per = dmap(lambda ev: reduce(smart_combine,
                                    detvalues(dmap(chain(over('layers', drop('loss')),
                                                         #on('layers', drop('0')),
                                                         only(['layers']), 
                                                         rec_liftup('layers'),
                                                         keymap(lambda k: k.replace('layers_','').replace('_accuracy', '')),
                                                         columnify()), 
                                                   ev))), 
                  pp_eds)
# Get everything into the correct columns
pers = list(sorted(la_for_per.keys()))
la_for_per = reduce(lambda p,c: liftup(c)(p),
                    la_for_per.iterkeys(),
                    la_for_per)
la_for_per = chain(columnify([e+'_type' for e in pers], 'Persoon', 'Laag'),
                   columnify([e+'_value' for e in pers], 'Persoon', 'Nauwkeurigheid'),
                   over('Persoon', lambda n: ''.join(n.split('_')[:-1])),
                   over('Persoon', anonymize))(la_for_per)
prettyjson(la_for_per)
# Plot!
sns.set_context('notebook', font_scale = 1.1)
p = sns.factorplot(data = dffd(la_for_per),
                   col = 'Laag',
                   y = 'Nauwkeurigheid',
                   x = 'Persoon',
                   kind = 'box',
                   size = 4,
                   aspect = 0.9,
                   order = list(sorted(set(la_for_per['Persoon']))))
#p.set_xticklabels(rotation=90)
sns.plt.suptitle('Nauwkeurigheid van autoencoderlagen per validatiestap')
sns.plt.tight_layout()
sns.plt.subplots_adjust(top=0.88)
show('encdecs-person-layer_accuracy')

In [40]:
# Not very useful
# Select data
la_for_per = dmap(lambda ev: reduce(smart_combine,
                                    detvalues(dmap(chain(only(['quality', 'train_quality']),
                                                         columnify()), 
                                                   ev))), 
                  pp_eds)
# Get everything into the correct columns
pers = list(sorted(la_for_per.keys()))
la_for_per = reduce(lambda p,c: liftup(c)(p),
                    la_for_per.iterkeys(),
                    la_for_per)
la_for_per = chain(columnify([e+'_type' for e in pers], 'Persoon', 'Resultaattype'),
                   columnify([e+'_value' for e in pers], 'Persoon', 'Waarde'),
                   over('Persoon', lambda n: ''.join(n.split('_')[:-1])))(la_for_per)
# Plot!
#p = sns.factorplot(data = dffd(la_for_per),
#                   col = 'Resultaattype',
#                   y = 'Waarde',
#                   x = 'Persoon',
#                   kind = 'box')
#p.set_xticklabels(rotation=30)
#sns.plt.show()

# Models

In [41]:
prettyjson(pe_mods)

In [ ]:
# Just label accuracy

In [13]:
# Label accuracy split out according to noise parameters
selector = 'accuracy'
y_naam = 'Nauwkeurigheid'
col      = 'drop_rate'
col_naam = 'Dropoutfrequentie'
pre_fun    = over('labels', only([selector]))
narrow_fun = only([col, 'labels'])
post_fun   = rec_liftup('labels')
rename     = keymap(lambda k: k.replace('labels_','').replace('_'+selector, '').replace(col, col_naam))
p = sns.factorplot(data = dffd(reduce(smart_combine, 
                                      detvalues(dmap(chain(pre_fun, narrow_fun, post_fun, rename), 
                                                     mods)))),
                   col = col_naam,
                   order = ['DISTRACT', 'RELAXOPEN', 'RELAXCLOSED', 'CASUAL', 'INTENSE'],
                   kind = 'box',
                   size = 5,
                   aspect = 0.6)
#sns.plt.ylabel(y_naam)
sns.plt.suptitle(y_naam+' per label i.f.v. '+col_naam)
p.set_xticklabels(rotation=45)
sns.plt.tight_layout()
sns.plt.subplots_adjust(top=0.90)
show('mods-'+col+'-'+selector)

In [14]:
# 1 result split out by 1 parameter
rename = lambda k: k.replace('gaussian_base_sigma','Sigma').replace('drop_rate','Dropfrequentie').replace('l2','L2-regularisatie').replace('epochs','Iteraties').replace('test_loss','Testverlies').replace('test_accuracy','Testnauwkeurigheid').replace('batch_size', 'Batchgrootte')
for x in ['gaussian_base_sigma', 'drop_rate', 'l2', 'epochs','batch_size']:
    selector = 'test_accuracy'
    kind     = 'box'
    p = sns.factorplot(data = dffd(reduce(smart_combine,detvalues(dmap(chain(only([selector, x]), keymap(rename)),mods)))),
                       x = rename(x),
                       y = rename(selector),
                       kind = kind,
                       size = 5)
    p.set_xticklabels(rotation=30)
    sns.plt.title(rename(selector)+' i.f.v. '+rename(x))
    sns.plt.tight_layout()
    show("mods-"+x+"-"+selector+"-"+kind)

In [44]:
# Are these useful?
# Label accuracy split out according to 2 parameters
for row in ['gaussian_base_sigma', 'drop_rate', 'l2', 'epochs']:
    for col in ['gaussian_base_sigma', 'drop_rate', 'l2', 'epochs']:
        if False: #row != col:
            selector = 'accuracy'
            outputs  = map((lambda l: l+'_'+selector), ['test','DISTRACT', 'RELAXOPEN', 'RELAXCLOSED', 'CASUAL', 'INTENSE'])
            kind     = 'box'
            p = sns.factorplot(data = dffd(reduce(smart_combine,
                                                  detvalues(dmap(chain(rec_liftup('labels'),
                                                                       keymap(lambda k: k.replace('labels_','')),
                                                                       only([row, col]+outputs)),
                                                                 mods)))),
                               row = row,
                               col = col,
                               order = outputs,
                               kind = kind,
                               size = 3,
                               aspect = 0.9)
            p.set_xticklabels(rotation=30)
            sns.plt.tight_layout()
            sns.plt.subplots_adjust(wspace=0.04)
            show("mods-"+row+"-vs-"+col+"-"+selector+"-"+kind)

/usr/lib/python2.7/site-packages/matplotlib/pyplot.py:516: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


In [45]:
# Accuracies (reg)pairplot
outputs  = ['Algemeen','DISTRACT', 'RELAXOPEN', 'RELAXCLOSED', 'CASUAL', 'INTENSE']
p = sns.pairplot(data = dffd(reduce(smart_combine,
                                    detvalues(dmap(chain(rec_liftup('labels'),
                                                         keymap(lambda k: k.replace('labels_', '').replace('_accuracy','').replace('test', 'Algemeen')),
                                                         only(outputs)),
                                                   mods)))),
                 vars = outputs,
                 kind = 'reg',
                 size = 2,
                 aspect = 1)
sns.plt.suptitle('Correlatie van nauwkeurigheden met elkaar')
sns.plt.tight_layout()
sns.plt.subplots_adjust(wspace = 0.09, hspace = 0.09, top = 0.96)
show('mods-accuracies-pairplot-reg')

In [12]:
# Label accuracy per experiment
# Select data
la_for_mod = dmap(lambda ev: reduce(smart_combine,
                                    detvalues(dmap(chain(over('labels', only(['accuracy'])),
                                                         only(['labels']), 
                                                         rec_liftup('labels'),
                                                         keymap(lambda k: k.replace('labels_', '').replace('_accuracy', '')),
                                                         columnify()), 
                                                   ev))), 
                  pe_mods)
# Get everything into the correct columns
la_for_mod = keymap(lambda s: '-'.join(s.split('-')[2:]), la_for_mod)
exps = list(sorted(la_for_mod.keys()))
la_for_mod = reduce(lambda p,c: liftup(c)(p),
                    la_for_mod.iterkeys(),
                    la_for_mod)
la_for_mod = chain(columnify([e+'_type' for e in exps], 'Experiment', 'Label'),
                   columnify([e+'_value' for e in exps], 'Experiment', 'Nauwkeurigheid'),
                   over('Experiment', lambda n: ''.join(n.split('_')[:-1])))(la_for_mod)
# Plot!
p = sns.factorplot(data = dffd(la_for_mod),
                   col = 'Label',
                   col_order = ['DISTRACT', 'RELAXOPEN', 'RELAXCLOSED', 'CASUAL', 'INTENSE'],
                   y = 'Nauwkeurigheid',
                   x = 'Experiment',
                   size = 4,
                   aspect = 0.9,
                   kind = 'box')
p.set_xticklabels(rotation=75)
sns.plt.suptitle('Nauwkeurigheid per label per experiment')
sns.plt.tight_layout()
sns.plt.subplots_adjust(top=0.88, bottom=0.39)
show('mods-experiment-label_accuracy')

In [56]:
# Label accuracy per person
# Select data
la_for_mod = dmap(lambda ev: reduce(smart_combine,
                                    detvalues(dmap(chain(over('labels', only(['accuracy'])),
                                                         only(['labels']), 
                                                         rec_liftup('labels'),
                                                         keymap(lambda k: k.replace('labels_', '').replace('_accuracy','')),
                                                         columnify()), 
                                                   ev))), 
                  pp_mods)
# Get everything into the correct columns
pers = list(sorted(la_for_mod.keys()))
la_for_mod = reduce(lambda p,c: liftup(c)(p),
                    la_for_mod.iterkeys(),
                    la_for_mod)
la_for_mod = chain(columnify([e+'_type' for e in pers], 'Persoon', 'Label'),
                   columnify([e+'_value' for e in pers], 'Persoon', 'Nauwkeurigheid'),
                   over('Persoon', lambda n: ''.join(n.split('_')[:-1])),
                   over('Persoon', anonymize))(la_for_mod)
# Plot!
p = sns.factorplot(data = dffd(la_for_mod),
                   col = 'Label',
                   col_order = ['DISTRACT', 'RELAXOPEN', 'RELAXCLOSED', 'CASUAL', 'INTENSE'],
                   y = 'Nauwkeurigheid',
                   x = 'Persoon',
                   size = 4,
                   aspect = 0.9,
                   kind = 'box',
                   order = list(sorted(set(la_for_mod['Persoon']))))
#p.set_xticklabels(rotation=90)
sns.plt.suptitle('Nauwkeurigheid per label per validatiestap')
sns.plt.tight_layout()
sns.plt.subplots_adjust(top=0.88)
#sns.plt.show()
show('mods-person-label_accuracy')

In [65]:
# Useful? Perhaps
# Select data
la_for_per = dmap(lambda ev: reduce(smart_combine,
                                    detvalues(dmap(chain(only(['test_accuracy']),
                                                         columnify()), 
                                                   ev))), 
                  pp_mods)
# Get everything into the correct columns
pers = list(sorted(la_for_per.keys()))
la_for_per = reduce(lambda p,c: liftup(c)(p),
                    la_for_per.iterkeys(),
                    la_for_per)
la_for_per = chain(columnify([e+'_type' for e in pers], 'Persoon', 'Resultaattype'),
                   columnify([e+'_value' for e in pers], 'Persoon', 'Nauwkeurigheid'),
                   over('Persoon', lambda n: ''.join(n.split('_')[:-1])),
                   over('Persoon', anonymize))(la_for_per)
# Plot!
#p = sns.factorplot(data = dffd(la_for_per),
#                   y = 'Nauwkeurigheid',
#                   x = 'Persoon',
#                   kind = 'box',
#                   order = list(sorted(set(la_for_per['Persoon']))))
#sns.plt.show()

# Cross-type

In [11]:
quality_vs_accuracy = {'Autoencoder kwaliteit': map(get('quality'), detvalues(eds))
                      #,'Autoencoder trainkwaliteit': map(get('train_quality'), detvalues(eds))
                      ,'Model nauwkeurigheid': map(get('test_accuracy'), detvalues(mods))}
quality_vs_accuracy = dict(quality_vs_accuracy, 
                           **reduce(smart_combine, 
                                    detvalues(dmap(chain(over('layers', chain(only(['accuracy']), 
                                                                              combine(['accuracy'], iden, 'nauwkeurigheid'))),
                                                         only(['layers']),
                                                         combine(['layers'], iden, 'Laag'),
                                                         rec_liftup('Laag'),
                                                         keymap(lambda k: k.replace('_',' '))), 
                                                   eds))))
sns.pairplot(data = dffd(quality_vs_accuracy),
             vars = ('Model nauwkeurigheid'
                    #,'Autoencoder kwaliteit'
                    #,'Autoencoder trainkwaliteit'
                    ,'Laag 0 nauwkeurigheid'
                    ,'Laag 1 nauwkeurigheid'
                    ,'Laag 2 nauwkeurigheid'),
             size = 2.5,
             aspect = 1,
             kind = 'reg')
sns.plt.suptitle('Correlatie van nauwkeurigheden van autoencoderlagen met modelnauwkeurigheid')
sns.plt.tight_layout()
sns.plt.subplots_adjust(wspace = 0.09, hspace = 0.09, top = 0.96)
show('cross-model-vs-layer_accuracy-pair')

In [75]:
# Maybe not very useful
quality_vs_accuracy = {'Autoencoder kwaliteit': map(get('quality'), detvalues(eds))}
                      #,'Autoencoder trainkwaliteit': map(get('train_quality'), detvalues(eds))}
quality_vs_accuracy = dict(quality_vs_accuracy, 
                           **reduce(smart_combine, 
                                    detvalues(dmap(chain(over('labels', chain(only(['accuracy']), 
                                                                              combine(['accuracy'], iden, 'nauwkeurigheid'))),
                                                         only(['labels']),
                                                         rec_liftup('labels'),
                                                         keymap(lambda k: k.replace('labels_', ''))), 
                                                   mods))))
#sns.pairplot(data = dffd(quality_vs_accuracy),
#             vars = ('Autoencoder kwaliteit'
#                    #,'Autoencoder trainkwaliteit'
#                    ,'DISTRACT_nauwkeurigheid'
#                    ,'RELAXOPEN_nauwkeurigheid'
#                    ,'RELAXCLOSED_nauwkeurigheid'
#                    ,'CASUAL_nauwkeurigheid'
#                    ,'INTENSE_nauwkeurigheid'))
#sns.plt.show()

In [77]:
# Not useful
lay_num = '2'
quality_vs_accuracy = {'Autoencoder kwaliteit': map(get('quality'), detvalues(eds))
                      ,'Autoencoder trainkwaliteit': map(get('train_quality'), detvalues(eds))
                      ,'Laag '+lay_num+' nauwkeurigheid': map(chain(get('layers'), get(lay_num), get('accuracy')), detvalues(eds))}
quality_vs_accuracy = dict(quality_vs_accuracy, 
                           **reduce(smart_combine, 
                                    detvalues(dmap(chain(over('labels', chain(only(['accuracy']), 
                                                                              combine(['accuracy'], iden, 'nauwkeurigheid'))),
                                                         only(['labels']),
                                                         rec_liftup('labels'),
                                                         keymap(lambda k: k.replace('labels_', ''))), 
                                                   mods))))
#data = dffd(quality_vs_accuracy)
#for lab in ['DISTRACT', 'RELAXOPEN', 'RELAXCLOSED', 'CASUAL', 'INTENSE']:
#    sns.kdeplot(data  = data['Laag '+lay_num+' nauwkeurigheid'],
#                data2 = data[lab+'_nauwkeurigheid'])
#    sns.plt.show()
#    sns.plt.clf()

In [78]:
# not useful
from_eds = {'Autoencoder sigma': map(chain(combine(['gaussian_base_sigma','enc_use_noise'], 
                                                   lambda bs, un: bs if un else 0,
                                                   'gaussian_base_sigma'),
                                           get('gaussian_base_sigma')),
                                     #filter(lambda d: not (d['enc_use_noise'] and d['gaussian_sigma_factor'] > 1), 
                                            detvalues(eds))}#)}
from_mods = {'Classifier sigma': map(get('gaussian_base_sigma'), detvalues(mods))}
                                     #detvalues(sift(lambda k,v: not (eds[k[:-2]]['enc_use_noise'] and eds[k[:-2]]['gaussian_sigma_factor'] > 1),
                                                    #items = True)(mods)))}
from_mods['Model nauwkeurigheid'] = map(get('test_accuracy'), detvalues(mods))
                                        #detvalues(sift(lambda k,v: not (eds[k[:-2]]['enc_use_noise'] and eds[k[:-2]]['gaussian_sigma_factor'] > 1),
                                                    #items = True)(mods)))
#data = dffd(dict(from_eds, **from_mods))
#sns.pairplot(data = data)
#sns.plt.show()

# Confusion

In [10]:
counts = dmap(chain(get('labels'), dmap(get('counts'))), mods)
train_size = dmap(lambda d: sum(i for v in d.values() for i in v), counts)
labels = ['DISTRACT', 'RELAXOPEN', 'RELAXCLOSED', 'CASUAL', 'INTENSE']
c = lambda mod,x,y: counts[mod][x][y]
li = lambda l: labels.index(l)
lr = lambda: xrange(len(labels))
# confusion counts
cc = {k: {l: [    c(k,l,li(l))
             ,sum(c(k,l,i) for i in lr() if i != li(l))
             ,sum(c(k,j,li(l)) for j in labels if j != l)
             ,sum(c(k,j,i) for i in lr() if i != li(l) for j in labels if j != l)]
          for l in labels}
      for k in counts.iterkeys()}
pzd = lambda n: 1 if n == 0 else n
confusions = {k: {'counts': cc[k]
                 ,'condition_rates':  {l: [float(cc[k][l][0])/pzd(cc[k][l][0]+cc[k][l][1])
                                          ,float(cc[k][l][1])/pzd(cc[k][l][1]+cc[k][l][0])
                                          ,float(cc[k][l][2])/pzd(cc[k][l][2]+cc[k][l][3])
                                          ,float(cc[k][l][3])/pzd(cc[k][l][3]+cc[k][l][2])]
                                       for l in labels}
                 ,'prediction_rates': {l: [float(cc[k][l][0])/pzd(cc[k][l][0]+cc[k][l][2])
                                          ,float(cc[k][l][1])/pzd(cc[k][l][1]+cc[k][l][3])
                                          ,float(cc[k][l][2])/pzd(cc[k][l][2]+cc[k][l][0])
                                          ,float(cc[k][l][3])/pzd(cc[k][l][3]+cc[k][l][1])]
                                       for l in labels}}
              for k in counts.iterkeys()}
prettyjson(confusions)

In [102]:
# Important condition rates
fig, (ax1, ax2) = sns.plt.subplots(ncols = 2, sharey = True)

# Sensitivity per label
sensitivities = dmap(chain(get('condition_rates'), dmap(get(0))),confusions)
sens_data = dffd(reduce(smart_combine, detvalues(sensitivities)))
p = sns.boxplot(data = sens_data, order = labels, ax=ax1)
p.set_xticklabels(labels, rotation = 30)
ax1.set_title('Sensitiviteit')
# Specificity per label
sensitivities = dmap(chain(get('condition_rates'), dmap(get(3))),confusions)
sens_data = dffd(reduce(smart_combine, detvalues(sensitivities)))
q = sns.boxplot(data = sens_data, order = labels, ax=ax2)
q.set_xticklabels(labels, rotation = 30)
ax2.set_title('Specificiteit')
sns.plt.suptitle('Statistieken voor binair gemaakte classificaties')
sns.plt.tight_layout()
sns.plt.subplots_adjust(top=0.90)
show('cond-sens_spec')

In [105]:
# Important prediction rates
fig, (ax1, ax2) = sns.plt.subplots(ncols = 2, sharey = True)

# Sensitivity per label
sensitivities = dmap(chain(get('prediction_rates'), dmap(get(0))),confusions)
sens_data = dffd(reduce(smart_combine, detvalues(sensitivities)))
p = sns.boxplot(data = sens_data, order = labels, ax=ax1)
p.set_xticklabels(labels, rotation = 30)
ax1.set_title('Positieve voorspellende waarde')
ax1.set_ylim([0.1,1])
# Specificity per label
sensitivities = dmap(chain(get('prediction_rates'), dmap(get(3))),confusions)
sens_data = dffd(reduce(smart_combine, detvalues(sensitivities)))
q = sns.boxplot(data = sens_data, order = labels, ax=ax2)
q.set_xticklabels(labels, rotation = 30)
ax2.set_title('Negatieve voorspellende waarde')
sns.plt.tight_layout()
show('pred-ppv_npv')

In [111]:
sensitivities = dmap(chain(get('condition_rates'), dmap(get(0))),confusions)
names = map(lambda k: '-'.join(k.split('-')[2:-2]), list(sorted(sensitivities.keys())))
sens_data = dffd(put('Naam',names)(reduce(smart_combine, detvalues(sensitivities))))
sns.pairplot(data = sens_data, vars = labels, hue = 'Naam')
sns.plt.suptitle('Correlatie van sensitiviteit van classificatie per label')
sns.plt.subplots_adjust(top=0.96, right=0.85, wspace=0.12, hspace=0.12)
show('cond-label-pair')

In [112]:
sensitivities = dmap(chain(get('prediction_rates'), dmap(get(0))),confusions)
names = map(lambda k: '-'.join(k.split('-')[2:-2]), list(sorted(sensitivities.keys())))
sens_data = dffd(put('Naam',names)(reduce(smart_combine, detvalues(sensitivities))))
sns.pairplot(data = sens_data, vars = labels, hue = 'Naam')
sns.plt.suptitle('Correlatie van PVW van classificatie per label')
sns.plt.subplots_adjust(top=0.96, right=0.85, wspace=0.12, hspace=0.12)
show('pred-label-pair')

In [121]:
sens_spec = dmap(chain(get('condition_rates'), dmap(get([0,2]))), confusions)
sens_spec = dmap(chain(lambda x: reduce(lambda p,c: liftup(c)(p), x.keys(), x), 
                       lambda x: columnify(sorted([k for k in x.keys() if '_0' in k]), 'Label', 'Echt positief')(x),
                       lambda x: columnify(sorted([k for k in x.keys() if '_1' in k]), 'Label', 'Vals positief')(x),
                       over('Label', lambda l: l.split('_')[0])), sens_spec)
sens_spec = reduce(lambda d, k: combine([dk for dk in d.iterkeys() if k+'-x' in dk], 
                                        lambda *i: reduce(smart_combine, i), 
                                        k)(d), 
                   set('-'.join(dk.split('-')[:-2]) for dk in sens_spec.iterkeys()),
                   sens_spec)
sens_spec = chain(lambda x: reduce(lambda p,c: liftup(c)(p), x.keys(), x), 
                  lambda x: columnify(sorted([k for k in x.keys() if '_Echt' in k]), 'Experiment', 'Echt positief')(x),
                  lambda x: columnify(sorted([k for k in x.keys() if '_Vals' in k]), 'Experiment', 'Vals positief')(x),
                  lambda x: columnify(sorted([k for k in x.keys() if '_Label' in k]), 'Experiment', 'Label')(x),
                  #over('Vals positief', lambda i: i*4),
                  over('Experiment', lambda l: '-'.join('_'.join(l.split('_')[:-1]).split('-')[2:])))(sens_spec)
ss_data = dffd(sens_spec).groupby(['Label','Experiment'], as_index = False).mean()
#print(ss_data.sort_values(by=['Label', 'Echt positief']))
fp = sns.lmplot(data = ss_data, 
                x = 'Vals positief',
                y = 'Echt positief',
                hue = 'Experiment',
                col = 'Label',
                col_order = labels,
                fit_reg = False,
                markers = 'o',
                size = 4,
                aspect = 0.9)
#import math
#fp.set(xticks=list(xrange(0, len(ss_data['Vals positief']), 50)))
#fp.set_xticklabels(rotation=30)
#mpp.plot([0,0.2], [0,1])
sns.plt.suptitle('ROC diagrammen per label met punten voor verschillende experimenten')
sns.plt.tight_layout()
sns.plt.subplots_adjust(top = 0.88, right = 0.90, wspace = 0.12)
show('roc-label-experiment')

In [12]:
# Cross-validation accuracies
# NOTA BENE: the next line contains very dirty code that only works if short-ratevar is the only multisetting experiment
per_params = keymap(lambda k: '-'.join(k.split('-')[:-2]+[str((lambda (i,j): i*2+j)(map(int, list(k.split('-')[-1]))[1:3]))]+[k.split('-')[-2]]), 
                    mods)
per_params = keymap(lambda k: k.replace('-0-','-') if not k.replace('-0-','-1-') in per_params else k)(per_params)
merged_crossval = reduce(lambda d, k: combine(sorted([dk for dk in d.iterkeys() if 'test-'+k+'-x' in dk]),
                                              lambda *i: reduce(smart_combine, i),
                                              k)(d),
                         set('-'.join(dk.split('-')[2:-1]) for dk in per_params),
                         dmap(get('test_accuracy'), per_params))
mcv_data = pd.concat(map(lambda (k,v): pd.DataFrame(v, columns=[k]), 
                         merged_crossval.iteritems()), 
                     join='outer', 
                     axis=1)
print(mcv_data.mean().sort_values())
mcv_data = mcv_data[list(reversed(mcv_data.median().sort_values().index))]
b = sns.boxplot(data=mcv_data)
b.set_xticklabels(mcv_data.columns, rotation=90)
sns.plt.title('Testnauwkeurigheid per experiment')
sns.plt.ylabel('Nauwkeurigheid')
sns.plt.tight_layout()
show('exp-accuracy')

on                 0.463375
shortest           0.471469
shorter            0.475933
short-ratevar-0    0.497992
eg_fd-l2           0.500853
eg_fd-nol2         0.509630
short-batch50      0.514925
short-ratevar-1    0.521719
long               0.547747
short-ratevar-2    0.561479
short              0.561784
off                0.563462
batch50-nogauss    0.565321
batch50            0.565926
short-ratevar-3    0.573999
batch50-nodrop     0.575166
dtype: float64


# Histories

In [126]:
for have_history in ['batch50', 'batch50-nogauss', 'batch50-nodrop', 'short-2', 'shorter'
                    ,'shortest', 'off-2', 'on-2', 'long-2', 'eg_fd-l2-2', 'eg_fd-nol2-2'
                    ,'short-batch50']:
    hf_names = ['final-test-'+have_history+'-x'+str(i)+'-0000000' for i in xrange(11)]
    historyfiles = ['ml/histories/'+hf_name+'.history.pkl.gz' for hf_name in hf_names]
    histories = list(xrange(11))
    for i in xrange(11):
        with gzip.open(historyfiles[i], 'rb') as f:
            histories[i] = cPickle.load(f)
    for history in histories:
        pd.Series(map(lambda (f, s): f, history)).plot()
    mpp.title(have_history.replace('-2','')+' verlies over iteraties')
    mpp.xlabel('Iteraties')
    mpp.ylabel('Verlies')
    mpp.tight_layout()
    show('loss-'+have_history)

In [128]:
for have_history in ['batch50', 'batch50-nogauss', 'batch50-nodrop', 'short-2', 'shorter'
                    ,'shortest', 'off-2', 'on-2', 'long-2', 'eg_fd-l2-2', 'eg_fd-nol2-2'
                    ,'short-batch50', 'short-ratevar']:
    hf_names = ['final-test-'+have_history+'-x'+str(i)+'-0000000' for i in xrange(11)]
    historyfiles = ['ml/histories/'+hf_name+'.history.pkl.gz' for hf_name in hf_names]
    histories = list(xrange(11))
    for i in xrange(11):
        with gzip.open(historyfiles[i], 'rb') as f:
            histories[i] = cPickle.load(f)
    # This is actually way too complicated... should keep it to a list...
    hist_data = pd.concat(map(lambda s: pd.DataFrame(data=map(lambda (l, a): l,s), 
                                                     index=list(xrange(len(s))),
                                                     columns=[hf_names[histories.index(s)][11:-8]]), 
                              histories), 
                          join='outer', 
                          axis=1)
    #print(hist_data.dropna(axis=0, how='any'))
    sns.tsplot(data = [hist_data.dropna(axis = 0, how = 'any')[col[11:-8]] for col in hf_names],
               ci = 90)
sns.plt.title('Gemiddeld verlies over iteraties met betrouwbaarheidsinterval')
sns.plt.xlabel('Iteraties')
sns.plt.ylabel('Verlies')
sns.plt.show()

In [129]:
max(len(l) for l in histories)

500

In [130]:
dffd(reduce(smart_combine, detvalues(chain(get('final-test-batch50-nodrop'), dmap(get('layers')), dmap(dmap(get('accuracy'))))(pe_eds))))

,0,1,2
0,0.823937,0.373714,0.230866
1,0.834895,0.380813,0.221216
2,0.836254,0.397414,0.221637
3,0.856275,0.332317,0.241980
4,0.853682,0.372477,0.281326
5,0.835938,0.339261,0.263938
6,0.826827,0.331562,0.240209
7,0.828349,0.383635,0.289162
8,0.841917,0.353006,0.214336
9,0.837860,0.339425,0.254350


In [131]:
chain(get('final-test-batch50-nodrop'), dmap(get('test_accuracy')))(pe_mods)

{u'x0-0000000': 0.5675616197183099,
 u'x1-0000000': 0.5574383802816901,
 u'x10-0000000': 0.5499031007751938,
 u'x2-0000000': 0.5928697183098591,
 u'x3-0000000': 0.5786713286713286,
 u'x4-0000000': 0.5701754385964912,
 u'x5-0000000': 0.5836267605633803,
 u'x6-0000000': 0.605241935483871,
 u'x7-0000000': 0.5712548638132295,
 u'x8-0000000': 0.5699823943661971,
 u'x9-0000000': 0.5801056338028169}

In [132]:
pe_eds.keys()

[u'final-test-on',
 u'final-test-eg_fd-l2',
 u'final-test-eg_fd-nol2',
 u'final-test-batch50',
 u'final-test-short-batch50',
 u'final-test-shortest',
 u'final-test-batch50-nodrop',
 u'final-test-short',
 u'final-test-shorter',
 u'final-test-batch50-nogauss',
 u'final-test-short-ratevar',
 u'final-test-off',
 u'final-test-long']

In [17]:
# per_m_params = keymap(lambda k: '-'.join(k.split('-')[:-2]+[str(sum(map(int, list(k.split('-')[-1]))))]+[k.split('-')[-2]]), 
#                       mods)

# per_e_params = keymap(lambda k: '-'.join(k.split('-')[:-2]+[str(sum(map(int, list(k.split('-')[-1]))))]+[k.split('-')[-2]]), 
#                       eds)
# NOTA BENE: the next lines contain very dirty code that only works if short-ratevar is the only multisetting experiment
per_m_params = keymap(lambda k: '-'.join(k.split('-')[:-2]+[str((lambda (i,j): i*2+j)(map(int, list(k.split('-')[-1]))[1:3]))]+[k.split('-')[-2]]), 
                      mods)
per_m_params = keymap(lambda k: k.replace('-0-','-') if not k.replace('-0-','-1-') in per_m_params else k)(per_m_params)
per_e_params = keymap(lambda k: '-'.join(k.split('-')[:-2]+[str((lambda (i,j): i*2+j)(map(int, list(k.split('-')[-1]))[1:3]))]+[k.split('-')[-2]]), 
                      eds)
per_e_params = keymap(lambda k: k.replace('-0-','-') if not k.replace('-0-','-1-') in per_e_params else k)(per_e_params)
m_merged_crossv = reduce(lambda d, k: combine(sorted([dk for dk in d.iterkeys() if 'test-'+k+'-x' in dk]),
                                              lambda *i: reduce(smart_combine, i),
                                              k)(d),
                         set('-'.join(dk.split('-')[2:-1]) for dk in per_m_params),
                         per_m_params)
e_merged_crossv = reduce(lambda d, k: combine(sorted([dk for dk in d.iterkeys() if 'test-'+k+'-x' in dk]),
                                              lambda *i: reduce(smart_combine, i),
                                              k)(d),
                         set('-'.join(dk.split('-')[2:-1]) for dk in per_e_params),
                         per_e_params)
def unionize(d1, d2):
    allkeys = set(list(d1.iterkeys())+list(d2.iterkeys()))
    def all_same_single(l):
        if all(li == l[0] for li in l):
            return l[0]
        else:
            return l
    def dep_comb(key, df, ds):
        if key in df and key in ds:
            if isinstance(df[key], list):
                return all_same_single(map(all_same_single,[df[key], ds[key]]))
            elif isinstance(df[key], dict):
                return unionize(df[key], ds[key])
            else:
                return smart_combine(df[key], ds[key])
        elif key in df:
            if isinstance(df[key], list):
                return all_same_single(df[key])
            else:
                return df[key]
        elif key in ds:
            if isinstance(ds[key], list):
                return all_same_single(ds[key])
            else:
                return ds[key]
        else:
            assert False
    return {k: dep_comb(k, d1, d2) for k in allkeys}
ultimate_d = dmap(drop(["pretrain_start_time"
                       ,"pretrain_end_time"
                       ,"l1"
                       ,"class_loss"
                       ,"encdecs_name"
                       ,"layer_sizes"
                       ,"class_optimizer"
                       ,"encdec_optimizer"]),unionize(e_merged_crossv, m_merged_crossv))
ultimate_d = keymap(lambda k: k[:-2] if not any(ok[:-2] == k[:-2] and not ok == k 
                                                for ok in ultimate_d.iterkeys()) else k)(ultimate_d)
ultimate_d = keymap(lambda k: k.replace('-2-','-'))(ultimate_d)
# print(dffd(dmap(chain(drop(["quality"
#                            ,"test_accuracy"
#                            ,"test_loss"
#                            ,"train_quality"
#                            ,"class_optimizer"
#                            ,"labels"
#                            ,"layers"
#                            ,"gaussian_sigma_factor"
#                            ,"enc_use_drop"]),
#                       on('drop_rate', lambda v: v if not isinstance(v,list) else max(v)),
#                       on('gaussian_base_sigma', lambda v: v if not isinstance(v,list) else max(v))))(ultimate_d)).T.to_latex())
print(dffd(dmap(chain(only(["test_accuracy",
                            "layers"]),
                      over("layers", chain(get("accuracy"), np.mean)),
                      on("test_accuracy", np.mean),
                      liftup("layers"),
                      keymap(lambda k: k.replace('layers_', 'Laag ').replace('test_accuracy','Classificatie'))))(ultimate_d)).T.to_latex())

\begin{tabular}{lrrrr}
\toprule
{} &  Classificatie &    Laag 0 &    Laag 1 &    Laag 2 \\
\midrule
                &       0.463375 &  0.661019 &  0.403223 &  0.603554 \\
batch           &       0.565926 &  0.838214 &  0.372515 &  0.237255 \\
batch50-nodr    &       0.575166 &  0.838009 &  0.357213 &  0.245829 \\
batch50-nogau   &       0.565321 &  0.847905 &  0.376463 &  0.436944 \\
eg\_fd-          &       0.500853 &  0.828707 &  0.361755 &  0.186883 \\
eg\_fd-no        &       0.509630 &  0.832286 &  0.358196 &  0.261973 \\
lo              &       0.547747 &  0.844222 &  0.382624 &  0.251670 \\
o               &       0.563462 &  0.836287 &  0.374702 &  0.368562 \\
sho             &       0.561784 &  0.813482 &  0.375978 &  0.296937 \\
short           &       0.475933 &  0.768737 &  0.347363 &  0.309382 \\
short-batch     &       0.514925 &  0.831583 &  0.369848 &  0.255335 \\
short-ratevar-0 &       0.497992 &  0.818342 &  0.345722 &  0.342383 \\
short-ratevar-1 &       0.521719 &

In [ ]:
#prettyjson(per_m_params)
print(set('-'.join(dk.split('-')[2:-1]) for dk in per_params))

In [ ]:
prettyjson(res)

In [ ]:
from datetime import datetime
times = {k: {"start": pe_mods[k]['x0-0000000']['pretrain_start_time']
            ,"end": res[k][max(drop(['cross_validation','settings'])(res[k]).iterkeys())]['end_time']}
         for k in res.iterkeys()}
times = dmap(dmap(lambda s: datetime.strptime(s, '%y%m%d-%H%M%S')), times)
dmap(chain(combine(['start', 'end'], lambda s,e: e-s, 'time'),get('time'),lambda td: td.total_seconds()), times)

In [39]:
# Requires confusions from previous block!
per_c_params = keymap(lambda k: '-'.join(k.split('-')[:-2]+[str((lambda (i,j): i*2+j)(map(int, list(k.split('-')[-1]))[1:3]))]+[k.split('-')[-2]]), 
                      confusions)
per_c_params = keymap(lambda k: k.replace('-0-','-') if not k.replace('-0-','-1-') in per_c_params else k)(per_c_params)
c_merged_crossv = reduce(lambda d, k: combine(sorted([dk for dk in d.iterkeys() if 'test-'+k+'-x' in dk]),
                                              lambda *i: reduce(smart_combine, i),
                                              k)(d),
                         set('-'.join(dk.split('-')[2:-1]) for dk in per_c_params),
                         per_c_params)
prettyjson(dmap(chain(over('counts', lambda l: map(np.mean,[l[i::4] for i in xrange(4)])),
                      get('counts')),c_merged_crossv))

In [49]:
confus_m = dmap(chain(get('labels'),
                      dmap(chain(get('counts'),
                                 lambda l: map(chain(np.mean,np.round), [l[i::5] for i in xrange(5)])))),
                m_merged_crossv)
print(dffd(confus_m['batch50-nodrop'])[labels].to_latex())

\begin{tabular}{lrrrrr}
\toprule
{} &  DISTRACT &  RELAXOPEN &  RELAXCLOSED &  CASUAL &  INTENSE \\
\midrule
0 &    7200.0 &      289.0 &        191.0 &   869.0 &    985.0 \\
1 &     355.0 &     7786.0 &        348.0 &   813.0 &    782.0 \\
2 &     201.0 &      273.0 &       7978.0 &   549.0 &    421.0 \\
3 &     606.0 &      359.0 &        261.0 &  4688.0 &   2265.0 \\
4 &     656.0 &      310.0 &        240.0 &  2099.0 &   4565.0 \\
\bottomrule
\end{tabular}



In [23]:
per_params = keymap(lambda k: '-'.join(k.split('-')[:-2]+[str((lambda (i,j): i*2+j)(map(int, list(k.split('-')[-1]))[1:3]))]+[k.split('-')[-2]]), 
                    mods)
per_params = keymap(lambda k: k.replace('-0-','-') if not k.replace('-0-','-1-') in per_params else k)(per_params)
merged_crossval = reduce(lambda d, k: combine(sorted([dk for dk in d.iterkeys() if 'test-'+k+'-x' in dk]),
                                              lambda *i: reduce(smart_combine, i),
                                              k)(d),
                         set('-'.join(dk.split('-')[2:-1]) for dk in per_params), 
                         per_params)
chain(get('labels'), dmap(chain(get('accuracy'), np.mean, lambda i: round(i,4))))(merged_crossval['batch50-nodrop'])

{u'CASUAL': 0.5179,
 u'DISTRACT': 0.7987,
 u'INTENSE': 0.5059,
 u'RELAXCLOSED': 0.885,
 u'RELAXOPEN': 0.8633}